In [419]:
import pandas as pd
import numpy as np
import re
import sys

# np.set_printoptions(threshold=sys.maxsize)

In [420]:
inputFile = 'data_formatted.csv'
outputFile = 'data_formatted_2.csv'

In [421]:
#read
df = pd.read_csv(inputFile, sep=',')

In [422]:
def split(columnName, seperator, newName = None, dropNaN=True):
    if newName == None:
        newName = columnName
    newFrame = []
    for _, row in df.iterrows():
        _id = row['id']
        item = str(row[columnName])
        if dropNaN and item == "nan":
            continue
        pieces = item.split(seperator)
        for piece in pieces:
            newFrame.append([_id, piece.strip()])
    return pd.DataFrame(newFrame, columns=['id', newName])

In [423]:
illegalCharsRegex = '[^üéöíøóäñÜÖßáÄ¢Šç’´‐\w\'\-\s.,]'
def findIllegalNames(data, column, regex):
    copy = data[column].dropna()
    return copy[copy.str.contains(regex, regex=True, flags= re.I)]

In [424]:
df

,Unnamed: 0,id,type,title,authors,editors,oldDepartmentNames,publisher,journal,booktitle,...,issue,vol,year,place,conference,Serie,link,comment,keywordsAndPeerReview,DOI
0,0,1,inbook,Wer sind die Experten? User Needs Analysis (UN...,"Linneweber, V.","Harloff, H.J.",Library,Verlag für Angewandte Psychologie,NaN,NaN,...,NaN,NaN,1993,Stuttgart,NaN,Psychologie des Wohnungs- und Siedlungbaus: Ps...,NaN,NaN,NaN,NaN
1,1,2,confpaper,Impact of climate change on forests: applicati...,"Lindner, M.; Bugmann, H.; Cramer, W.; Lasch, P.","Vancura, K.; Sramek, V.",Global Change & Natural Systems,Forestry and Game Management Research Institute,NaN,NaN,...,NaN,NaN,1999,Prague,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,lecture,Quantifizierung des Einflusses verschiedener (...,"Bronstert, A.",NaN,Global Change & Natural Systems,NaN,NaN,NaN,...,NaN,NaN,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,confpaper,Ein Modell der Sukzessionsdynamik europäischer...,"Schaber, J.; Badeck, F.-W.; Lasch, P.","Pelz, D.R.; Rau, O.; Saborowski, J.",Global Change & Natural Systems,Deutscher Verband forstlicher Versuchsanstalte...,NaN,NaN,...,NaN,NaN,1999,Freiburg,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,confpaper,Comparison of data requirements and performanc...,"Krysanova, V.; Bronstert, A.; Menzel, L.",NaN,Global Change & Natural Systems,Dt. Nationalkomitee f. d. IHP der UNESCO u. d....,NaN,NaN,...,NaN,NaN,1999,Koblenz,NaN,NaN,NaN,NaN,NaN,NaN
5,5,6,paperr,Delta 13C of CO2 respired in the dark in relat...,"Duranceau, M.; Ghashghaie, J.; Badeck, F.-W.; ...",NaN,Global Change & Natural Systems,NaN,"Plant, Cell and Environment",NaN,...,5,22,1999,NaN,NaN,NaN,NaN,Nat Sys,Phaseolus vulgaris; carbohydrates; carbon isot...,NaN
6,6,7,inbook,Land use change in Europe and implications for...,"Krysanova, V.; Wechsung, F.; Meiner, A.; Vasil...","Ennuste, Ü.; Wilder, L.",Global Change & Natural Systems,"Estonian Institute of Economics, Tallinn TU",NaN,Harmonization with the western economics: Esto...,...,NaN,NaN,1999,Tallinn,NaN,Harmonization with the western economics: Esto...,NaN,NaN,NaN,NaN
7,7,8,confpaper,Großräumige hydrologische Parametrisierung und...,"Bronstert, A.; Güntner, A.; Jaeger, A.; Krol, ...","Fohrer, N.; Döll, P.","Global Change & Natural Systems, Integrated Sy...",Kassel University Press,NaN,NaN,...,NaN,NaN,1999,Rauischholzhausen,NaN,NaN,NaN,3-933146-21-6,NaN,NaN
8,8,9,confpaper,The use of semi-qualitative reasoning and prob...,"Moldenhauer, O.; Bruckner, T.; Petschel-Held, G.",NaN,Integrated Systems Analysis,PIK; IOS Press,NaN,NaN,...,NaN,NaN,1999,Amsterdam,NaN,NaN,NaN,Int. Systemanalyse/ICLIPS; QUESTIONS,Qualitative Reasoning; QSIM; Semiquantitative ...,NaN
9,9,10,confpaper,Effects of climate change influencing storm ru...,"Bronstert, A.; Bürger, G.; Heidenreich, M.; Ka...","Casale, R.; Samuels, P.; Bronstert, A.","Global Change & Natural Systems, Climate Research",Office for Official Publications of the Europe...,NaN,NaN,...,NaN,NaN,1999,Luxemburg,NaN,NaN,NaN,NaN,NaN,NaN


In [425]:
#shortens name: Lastname, F.S.
def shortenNames(columnName):
    column = df[columnName] #.dropna()
    #print(column)
    
    wrong = []

    for x in range(len(column)):
        end = []
        
        #checkes whether value is nan
        if column[x] != column[x]:
            continue
            
        names = column[x].split('; ')
        for i in range(len(names)):
            ns = names[i].split(', ')
            
            #ns has to have two parts: lastname, firstname
            if len(ns) == 1:
                wrong.append([x, i])
                end.append(names[i])
                continue
            #print(ns)
            firstname = ns[1]
            lastname = ns[0]
           
            #checkes whether the value alreaddy has short parts
            if (re.search('.\.', firstname)):
                newFN = firstname.replace('-', '') #.replace(' ', '')
                firstnames = newFN.split(' ')
                    
                fn = ''
                for j in range(len(firstnames)):
                    if firstnames[j] == '':
                        fn += ''
                        continue
                    if (re.search('.\.', firstnames[j])):
                        fn += firstnames[j]
                    else:
                        #first letter of name
                        fn += firstnames[j][0] + '.'
                end.append(lastname + ', ' + fn)
            else:
                newFN = firstname.replace('-', ' ') #.replace(' ', '')
                firstnames = newFN.split(' ')
                fn = ''
                for j in range(len(firstnames)):
                    if firstnames[j] == '':
                        fn += ''
                        continue
                    if len(firstnames[j]) == 1:
                        fn += firstnames[j] + '.'
                    else:
                        #first letter of name
                        fn += firstnames[j][0] + '.'
                end.append(lastname + ', ' + fn) 

In [426]:
shortenNames('authors')

0
Linneweber, V.
1
Lindner, M.; Bugmann, H.; Cramer, W.; Lasch, P.
2
Bronstert, A.
3
Schaber, J.; Badeck, F.W.; Lasch, P.
4
Krysanova, V.; Bronstert, A.; Menzel, L.
5
Duranceau, M.; Ghashghaie, J.; Badeck, F.W.; Deleens, E.C.
6
Krysanova, V.; Wechsung, F.; Meiner, A.; Vasiljev, A.
7
Bronstert, A.; Güntner, A.; Jaeger, A.; Krol, M.; Krywkow, J.
8
Moldenhauer, O.; Bruckner, T.; Petschel-Held, G.
9
Bronstert, A.; Bürger, G.; Heidenreich, M.; Katzenmaier, D.K.
10
Bronstert, A.; Ghazi, A.; Hladny, J.; Kundzewicz, Z.
11
Sprinz, D.
12
Gräfe, J.; Poschenrieder, W.; Wechsung, F.; Kartschall, T.
13
Lahmer, W.; Becker, A.
14
Lahmer, W.
15
Lahmer, W.; Becker, A.
16
Lahmer, W.; Klöcking, B.; Pfützner, B.
17
Schellnhuber, H.J.; Enke, W.; Flechsig, M.
18
Krysanova, V.; Gerten, D.; Klöcking, B.; Becker, A.
19
Cramer, W.; Kicklighter, D.W.; Bondeau, A.; Moore, B.; Nemry, B.; Ruimy, A.; Schloss, A.
20
Cramer, W.
21
Bronstert, A.; Kundzewicz, Z.W.; Menzel, L.
22
Müller-Wohlfeil, D.I.; Lahmer, W.; Krysano

Toth, F.
537
Krysanova, V.
538
Toth, F.
539
Cramer, W.
540
Cramer, W.
541
Toth, F.
542
Reusswig, F.
543
Cramer, W.
544
Toth, F.
545
Bugmann, H.
546
Bronstert, A.
547
Toth, F.
548
Böhm, U.
549
Krysanova, V.; Tenno, K.
550
Grote, R.
551
Nakicenovic, N.; Nordhaus, W.D.; Richels, R.; Toth, F.L.
552
Werner, P.C.; Böhm, U.; Gerstengarbe, F.W.
553
Grote, R.; Suckow, F.; Bellmann, K.
554
Müller-Wohlfeil, D.I.; Lahmer, W.; Pfützner, B.; Becker, A.
555
Müller-Wohlfeil, D.I.; Krysanova, V.; Becker, A.
556
Svirezhev, Y.M.
557
Helm, C.
558
Bronstert, A.
559
Bronstert, A.
560
Becker, A.
561
Woodward, F.I.; Cramer, W.
562
Steffen, W.L.; Cramer, W.; Plöchl, M.; Bugmann, H.
563
Lindner, M.; Lasch, P.; Cramer, W.
564
Werner, P.C.; Gerstengarbe, F.W.
565
Bronstert, A.; Plate, E.J..
566
Bronstert, A.
567
Lahmer, W.; Becker, A.
568
Bronstert, A.
569
Bronstert, A.
570
Müller-Wohlfeil, D.I.; Krysanova, V.
571
Bronstert, A.
572
Sprinz, D.
573
Toth, F.
574
Lahmer, W.; Müller-Wohlfeil, D.I.; Becker, A.
575
Krys

1043
Krol, M.; Bronstert, A.
1044
Krysanova, V.; Gerten, D.; Klöcking, B.; Becker, A.
1045
Toth, F.
1046
Helm, C.
1047
Prentice, I.C.; Cramer, W.; Harrison, S.P.; Leemans, R.M.; Solomon, A.M.
1048
Helm, C.
1049
Helm, C.
1050
Stock, M.; Schellnhuber, H.J.
1051
Wechsung, F.; Krysanova, V.; Flechsig, M.; Schapoff, S.
1052
Kropp, J.; von Bloh, W.; Klenke, Th.
1053
Toth, F.
1054
Helm, C.
1055
Bronstert, A.
1056
Becker, A.; Lahmer, W.; Müller-Wohlfeil, D.I.; Pfützner, B.
1057
Wenzel, V.; Kücken, M.; Flechsig, M.
1058
Bondeau, A.; Kicklighter, D.W.; Kaduk, J.
1059
Lasch, P.; Bugmann, H.; Lindner, M.; Schaber, J.
1060
Helm, C.
1061
Bruschek, G.J.
1062
Helm, C.
1063
Wechsung, F.; Poschenrieder, W.; Krysanova, V.; Gräfe, J.; Kartschall, T.
1064
Wrobel, M.
1065
Franck, S.
1066
Fröhlich, A.; Erhard, M.; Gah, E.; Cramer, W.
1067
Stock, M.
1068
Bugmann, H.; Martin, Ph.
1069
Franck, S.
1070
Stock, M.
1071
Toth, F.
1072
Becker, A.; Lahmer, W.
1073
Bruckner, T.
1074
Schellnhuber, H.J.; Yohe, G.
1075
Fr

1551
Dauber, J.; Schroeter, D.; Wolters, V.
1552
Sprinz, D.F.
1553
Ekschmitt, K.; Baxter, R.; Campbell, C.; Ceulemans, R.; Günther, K.; Hector, H.; Henning, F.; Kjøller, A.; Matouch, S.; Molau, U.; Nagy, P.; Otte, A.; Pausas, J.; Pflug, A.; Schroeter, D.; Sillence, G.; Steiner, N.; Struwe, S.; Wolters, V.; Wookey, P.
1554
Breitung, W.; Chan, C.; Dorofeev, S.B.; Eder, A.; Gelfand, B.E.; Heitsch, M.; Klein, R.; Malliakos, A.; Shepherd, J.E.; Studer, E.; Thibault, P.
1555
Munz, C.D.; Roller, S.; Klein, R.; Geratz, K.J.
1556
Helling, C.; Oevermann, M.; Luettke, M.J.H.; Klein, R.; Sedlmayr, E.
1557
Klein, R.; Breitung, W.; Coe, I.; Grönig, H.; He, L.; Olivier, H.; Rehm, W.; Studer, E.; Vendel, J.
1558
Schneider, T.; Klein, R.
1559
Klein, R.; Botta, N.; Owinoh, A.
1560
Schmidt, H.; Klein, R.
1561
Klein, R.
1562
Oevermann, M.; Klein, R.; Berger, M.; Goodman, J.
1563
Stoll-Kleemann, S.
1564
Welp, M.
1565
Werner, P.C.; Gerstengarbe, F.W.; Fraedrich, K.; Oesterle, H.
1566
Fraedrich, K.; Gersteng

Rahmstorf, S.
2061
Solanki, S.; Ohmura, A.; Beer, J.; Fröhlich, C.; Latif, M.; Rahmstorf, S.; Schönwiese, C.; Neu, U.
2062
Rahmstorf, S.
2063
Rahmstorf, S.
2064
Rahmstorf, S.
2065
Klein, R.J.T.
2066
Klein, R.J.T.; Smith, J.B.
2067
Smith, J.B.; Klein, R.J.T.; Huq, S.
2068
Sterr, H.; Klein, R.J.T.; Reese, S.
2069
Runge, D.
2070
Dirmeyer, P.; Hoff, H.
2071
Hoff, H.(.a.; Kabat, P.; Bates, B.; Hellmuth, M.; Bullock, A.; van Schaik, H.; Connor, R.
2072
Pahl-Wostl, C.; Hoff, H.; Meybeck, M.; Sorooshian, S.
2073
Hoff, H.
2074
Hoff, H.; Jaeger, C.; Leveque, C.; Vorosmarty, C.
2075
Hunt, J.C.R.; Orr, A.; Cresswell, D.; Owinoh, A.
2076
Thonicke, K.; Venevsky, S.; Cramer, W.
2077
Thonicke, K.; Cramer, W.
2078
Thonicke, K.
2079
Alcamo, J.; Klein, R.J.T.; Carius, A.; Acosta-Michlik, L.; Krömker, K.; Tänzler, D.; Eierdanz, F.
2080
Füssel, H.M.; Klein, R.J.T.
2081
Tol, R.S.J.; Klein, R.J.T.; Nicholls, R.J.
2083
Klein, R.J.T.; Schipper, E.L.; Dessai, S.
2084
Hasselmann, K.; Latif, M.; Hooss, G.; Azar, 

...

Acharya, T.; Fanzo, J.; Gustafson, D.; Ingram, J.; Schneeman, B.; Allen, L.; Boote, K.; Drewnowski, A.; Ewert, F.; Hall, S.; Hendley, P.; Howden, M.; Janssen, S.; Jones, J.; Latulippe, M.; Lotze-Campen, H.; McDermott, J.; Meijl, H.v.; Nelson, G.; Newsome, R.; Roulin, A.; Scholes, B.; Tanumihardjo, S.; Tavill, G.; Mensbrugghe, D.v.d.; Wiebe, K.
6430
Elliott, J.; Müller, C.; Deryng, D.; Chryssanthacopoulos, J.; Boote, K.J.; Büchner, M.; Foster, I.; Glotter, M.; Heinke, J.; Iizumi, T.; Izaurralde, R.C.; Mueller, N.D.; Ray, D.K.; Rosenzweig, C.; Ruane, A.C.; Sheffield, J.
6431
Biewald, A.; Rolinski, S.
6432
Biewald, A.; Rolinski, S.
6433
Gornott, C.; Wechsung, F.
6434
Dangendorf, S.; Rybski, D.; Mudersbach, C.; Müller, A.; Kaufmann, E.; Zorita, E.; Jensen, J.
6435
Oijen, M.v.; Balkovič, J.; Beer, C.; Cameron, D.; Ciais, P.; Cramer, W.; Kato, T.; Kuhnert, M.; Martin, R.; Myneni, R.; Rammig, A.; Rolinski, S.; Soussana, J.F.; Thonicke, K.; Velde, M.v.d.; Xu, L.
6436
Yue, C.; Ciais, P.; Cadule

6702
Valli, D.; Muthuswamy, B.; Banerjee, S.; Ariffin, M.R.K.; Wahab, A.W.A.; Ganesan, K.; Subramaniam, C.K.; Kurths, J.
6703
Suresh, R.; Srinivasan, K.; Senthilkumar, D.V.; Murali, K.; Lakshmanan, M.; Kurths, J.
6704
Belykh, I.; di Bernardo, M.; Kurths, J.; Porfiri, M.
6705
Schinkel, S.; Ivanova, G.; Kurths, J.; Sommer, W.
6706
Hassan, M.K.; Pavel, N.I.; Pandit, R.K.; Kurths, J.
6707
Kapitaniak, T.; Kurths, J.
6708
Mahdavi, N.; Kurths, J.
6709
Peron, T.K.D.; Comin, C.H.; Amancio, D.R.; Costa, L.d.F.; Rodrigues, F.A.; Kurths, J.
6710
Tang, Y.; Qian, F.; Gao, H.; Kurths, J.
6711
Li, L.; Kurths, J.; Yang, Y.; Liu, G.
6712
Rehfeld, K.; Molkenthin, N.; Kurths, J.
6713
Battaglini, A.; Bätjer, S.
6714
Steffen, W.; Richardson, K.; Rockström, J.; Cornell, S.; Fetzer, I.; Bennett, E.M.; Biggs, R.; Carpenter, S.R.; de Vries, W.; de Wit, C.A.; Folke, C.; Gerten, D.; Heinke, J.; Mace, G.M.; Persson, L.M.; Ramanathan, V.; Reyers, B.; Sörlin, S.
6715
Klein, D.; Humpenöder, F.; Bauer, N.; Dietrich, J

7171
Boysen, M.
7172
Notbohm, A.; Kurths, J.; Herrmann, C.S.
7173
Körner, C.; Basler, D.; Hoch, G.; Kollas, C.; Lenz, A.; Randin, C.F.; Vitasse, Y.; Zimmermann, N.E.
7174
Levermann, A.; Petoukhov, V.; Schewe, J.; Schellnhuber, H.J.
7175
Afsar, O.; Tirnakli, U.; Kurths, J.
7176
Zou, Y.; Donner, R.V.; Thiel, M.; Kurths, J.
7177
Donges, J.F.; Schleussner, C.F.; Siegmund, J.F.; Donner, R.V.
7178
Kan, A.v.; Jegminat, J.; Donges, J.F.; Kurths, J.
7179
Rogozhina, I.; Petrunin, A.G.; Vaughan, A.P.M.; Steinberger, B.; Johnson, J.V.; Kaban, M.K.; Calov, R.; Rickers, F.; Thomas, M.; Koulakov, I.
7180
Bren d'Amour, C.; Wenz, L.; Kalkuhl, M.; Steckel, J.C.; Creutzig, F.
7181
Larondelle, N.; Strohbach, M.W.
7182
Kain, J.H.; Larondelle, N.; Haase, D.; Kaczorowska, A.
7183
Stadtherr, L.; Coumou, D.; Petoukhov, V.; Petri, S.; Rahmstorf, S.
7184
Aich, V.; Koné, B.; Hattermann, F.F.; Paton, E.N.
7185
Eugenio, E.A.; Acosta, L.A.; Enano, N.H.; Magcale-Macandog, D.B.; Macandog, P.B.M.; Talubo, J.P.P.; Salva

Blumenthal, I.; Schlenther, C.; Klinge, G.; Neis, R.
7467
Blumenthal, I.; Schlenther, C.; Klinge, G.; Neis, R.
7468
Blumenthal, I.; Schlenther, C.; Klinge, G.; Neis, R.
7469
Blumenthal, I.; Schlenther, C.; Klinge, G.; Neis, R.
7470
Blumenthal, I.; Schlenther, C.; Hirsbrunner, S.; Stock, M.; Nocke, T.
7471
Durand, J.L.; Delusca, K.; Boote, K.; Lizaso, J.; Manderscheid, R.; Weigel, H.J.; Ruane, A.C.; Rosenzweig, C.; Jones, J.; Ahuja, L.; Anapalli, S.; Baron, C.; Basso, B.; Bertuzzi, P.; Biernath, C.; Deryng, D.; Ewert, F.; Gaiser, T.; Gayler, S.; Heinlein, F.; Kersebaum, K.C.; Kim, S.H.; Müller, C.; Nendel, C.; Olioso, A.; Priesack, E.; Ramirez Villegas, J.; Ripoche, D.; Rötter, E.R.; Seidel, S.I.; Srivastava, A.; Tao, F.; Timlin, D.; Twine, T.; Wang, E.; Webber, H.; Zhao, Z.
7472
Hezel, B.
7473
Virgin, I.; Fielding, M.; Sundell, M.F.; Hoff, H.; Granit, J.
7474
Lie, X.; Liu, J.; Han, G.; Hoff, H.
7475
Bonzi, C.; Onigkeit, J.; Hoff, H.; Joyce, B.; Tielboerger, K.
7476
Bodirsky, B.
7477
St

Piniewski, M.; Marcinkowski, P.; Kundzewicz, Z.W.
7971
Siegmeier, J.; Mattauch, L.; Edenhofer, O.
7972
Blumenthal, I.; Schlenther, C.; Hirsbrunner, S.; Stock, M.; Nocke, T.
7973
Guimberteau, M.; Ciais, P.; Ducharne, A.; Boisier, J.P.; Dutra Aguiar, A.P.; Biemans, H.; De Deurwaerder, H.; Galbraith, D.; Kruijt, B.; Langerwisch, F.; Poveda, G.; Rammig, A.; Rodriguez, D.A.; Tejada, G.; Thonicke, K.; Von Randow, C.; Von Randow, R.C.S.; Zhang, K.; Verbeeck, H.
7974
Han, P.; Zeng, N.; Zhao, F.; Lin, X.
7975
Henrot, A.J.; Utescher, T.; Erdei, B.; Dury, M.; Hamon, N.; Ramstein, G.; Krapp, M.; Herold, N.; Goldner, A.; Favre, E.; Munhoven, G.; Francois, L.
7976
Jakob, M.; Kübler, D.; Steckel, J.C.; Veldhuizen, R.v.
7978
Jungclaus, J.H.; Bard, E.; Baroni, M.; Braconnot, P.; Cao, J.; Chini, L.P.; Egorova, T.; Evans, M.; González-Rouco, J.F.; Goosse, H.; Hurtt, G.C.; Joos, F.; Kaplan, J.O.; Khodri, M.; Goldewijk, K.K.; Krivova, N.; LeGrande, A.N.; Lorenz, S.J.; Luterbacher, J.; Man, W.; Maycock, A.C

In [120]:
authors = split('authors', ';', 'name')
findIllegalNames(authors, 'name', illegalCharsRegex)

90                                           Del‚colle, R.
1285                                  Gu‚rif, M. Delecolle
1294                                         Del‚colle, R.
1828                                         G‚rard, J.-C.
1957     IGBP Terrestrical Carbon Working Group (incl. ...
2351                 WASA Group (Carretero J.C.,  Gomes M.
3373                             Toth, F.L. (guest editor)
3773               et al. (including Schellnhuber,  H.-J.)
4215                     (in co-operation with Becker,  D.
4216                                      Ballerstedt, K.)
4404                                          ï¿½quist, M.
4462                                          Noguï¿½s, S.
5295                        Hoff, H. (contributing author)
7715                                          Kï¿½hler, J.
7809                                         FrÃ¶hlich, J.
7936                                        Klï¿½cking, B.
7937                                          Strï¿½bl, 

In [8]:
editors = split('editors', ';', 'name')
findIllegalNames(editors, 'name', illegalCharsRegex)

318                                            Sabat‚, S.
375                                           gren, G.I.
657     Ministerium für Landwirtschaft,  Umweltschutz ...
1017                                   ORiordan, T. (ed.)
Name: name, dtype: object

In [9]:
oldDepartmentNames = split('oldDepartmentNames', ',', 'name')
oldDepartmentNames['name'].unique()

array(['Library', 'Global Change & Natural Systems',
       'Integrated Systems Analysis', 'Climate Research',
       'Social Systems', 'Data & Computation', 'BAHC', 'Climate System'],
      dtype=object)

In [ ]:
df['type'].unique()

In [ ]:
df['publisher'].dropna().unique()

In [ ]:
df['journal'].dropna().unique()

In [ ]:
findIllegalNames(df, 'startpage', '[^0-9]').unique()

In [ ]:
findIllegalNames(df, 'endpage', '[^0-9]').unique()

In [ ]:
df['issue'].dropna().unique()
# findIllegalNames(df, 'issue', '[^0-9]').unique()

In [ ]:
df['vol'].dropna().unique()

In [ ]:
df['year'].dropna().unique()

In [ ]:
findIllegalNames(df, 'place', illegalCharsRegex)

In [ ]:
df['link'].dropna().unique()

# findIllegalNames(df, 'issue', '^$http.*').unique()

In [ ]:
df['comment'].dropna().unique()

In [ ]:
df['keywordsAndPeerReview'].dropna().unique()

In [ ]:
df['DOI'].dropna().unique()

In [ ]:
df.loc[df['journal'].dropna().index]['type'].unique()

In [ ]:
# booktitle appears not only in 'inbook' but also in other types of publication
df.loc[df['booktitle'].dropna().index]['type'].unique()

In [ ]:
# df[~df['relation (= Serie)'].isnull()]
df['Serie'].dropna().unique().shape

In [ ]:
df[~df['Serie'].isnull()]['type'].unique()

In [ ]:
df['conference'].dropna().unique().shape

In [ ]:
df.loc[df['conference'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['DOI'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['startpage'].dropna().index]['type'].unique()

In [427]:
df.to_csv(outputFile, encoding="utf8")